In [ ]:
#%run "./constants"

In [ ]:
import pandas as pd
from datetime import datetime
import glob

def concat_and_filter(
        download_dir: str = DIR_DOWNLOAD,
        class_dict: str = CLASS_DICT
) -> pd.DataFrame:
    
    """
        Concatenates and filters data.
        
        This function reads parquet files from a specified directory, filters the data based on a provided class dictionary, and returns a pandas DataFrame.

        Args:
            download_dir (str): The directory from where the parquet files will be read. The default value is DIR_DOWNLOAD.
            class_dict (str): A dictionary that maps class names to procedure codes. The default value is CLASS_DICT.

        Returns:
            pd.DataFrame: A pandas DataFrame that contains the filtered data.

        Example:
            Here is an example of how to use this function::

                df = concat_and_filter(download_dir="my_directory", class_dict={"class1": ["code1", "code2"]})
    """

    all_paths = [f'{path}' for file_path in glob.glob(f"{download_dir}/*.parquet") for path in glob.glob(f'{file_path}/*.parquet')]

    # Criando um DataFrame vazio para armazenar os resultados
    final_filtered_df = pd.DataFrame()
    print('Function Concat: Started')
    for file_path in all_paths:
        #print(file_path)
        df = pd.read_parquet(file_path)

        filtered_values = df[df['PA_PROC_ID'].isin([item for sublist in class_dict.values() for item in sublist])].copy()
        
        filtered_values['CLASSE'] = filtered_values.apply(lambda row: next((class_name for class_name, class_codes in class_dict.items() if row['PA_PROC_ID'] in class_codes), None), axis=1)

        # Adding to the empty DataFrame
        final_filtered_df = pd.concat([final_filtered_df, filtered_values], ignore_index=True)

    # Creating columns: DATA_DOWNLOAD and DATA_PROCESSAMENTO.
    #final_filtered_df['DATA_DOWNLOAD'] = datetime.now().strftime("%Y-%m-%d")
    final_filtered_df['DATA_DOWNLOAD'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    final_filtered_df['DATA_DOWNLOAD'] = pd.to_datetime(final_filtered_df['DATA_DOWNLOAD'])
    final_filtered_df['DATA_PROCESSAMENTO'] = pd.to_datetime(final_filtered_df['PA_MVM'], format='%Y%m')
    #final_filtered_df.to_csv(f'backup_datasus_{datetime.now().strftime("%d%m%y")}.csv', index=False)
    print('Function Concate has ended')
    return final_filtered_df

In [ ]:
dataframe = concat_and_filter()